In [2]:
import pandas as pd
import nltk 
from nltk.stem import WordNetLemmatizer
import gensim
from gensim.models import Doc2Vec
import pickle

In [3]:
df_NM= pd.read_csv("D:\FINAL_PM_Modi_Speech_Cleaned_string.csv")

In [4]:
df_NM.head()

,index,Date,Speech,Title,Cleaned_Speech,Cleaned_Speech_string
0,NMS1,2017-01-03,"Governor of Andhra Pradesh, Shri E. S. L. Nara...",PM's Address at the Inauguration of the 104th ...,['governor andhra pradesh shri e l narasimhan ...,governor andhra pradesh shri e l narasimhan ch...
1,NMS2,2017-01-09,Chief Minister of Gujarat Shri Vijay Rupani ji...,Text of PM's address at the Inauguration of No...,['chief minister gujarat shri vijay rupani ji ...,chief minister gujarat shri vijay rupani ji co...
2,NMS3,2017-01-09,I am delighted to be here at Gift City to ina...,Text of PM’s address on the occasion of Inaugu...,['delighted gift city inaugurate india first i...,delighted gift city inaugurate india first int...
3,NMS4,2017-02-01,I congratulate the Finance Minister Arun Jaitl...,English rendering of the Prime Minister’s Stat...,['congratulate finance minister arun jaitley j...,congratulate finance minister arun jaitley jee...
4,NMS5,2017-02-05,Sri Pejavar Math’s most respected Sri Vishwesh...,English rendering of the text of PM’s address ...,['sri pejavar math respected sri vishwesh tirt...,sri pejavar math respected sri vishwesh tirth ...


In [5]:
df_MS= pd.read_csv("D:\PM_MANMOHAN_Speech_Cleaned_string.csv")

In [6]:
df_MS = df_MS.drop('Location', axis=1)

In [7]:
df_MS.head()

,index,Date,Title,Speech,Cleaned_Speech,Cleaned_Speech_string
0,MSS1,2004-06-24,Prime Minister 's Address to the Nation,"My Fellow Citizens,I speak to you with a deep...",['fellow citizen speak deep sense humility ful...,fellow citizen speak deep sense humility fully...
1,MSS2,2004-06-29,PM's inaugural address at the Chief Ministers'...,"""I welcome you all to this Conference which w...",['welcome conference take stock effort rural d...,welcome conference take stock effort rural dev...
2,MSS3,2004-07-03,"PM's inaugural address at the ""Dialogue on Min...","""It gives me great pleasure to be amongst you...","['give great pleasure amongst morning', 'parti...",give great pleasure amongst morning particular...
3,MSS4,2004-07-26,PM's speech at the CSIR Society Meeting,"Distinguished Members of the CSIR Society, La...",['distinguished member csir society lady gentl...,distinguished member csir society lady gentlem...
4,MSS5,2004-07-31,PM’ statement at the Press Conference,"\n\t""My two days in Bangkok have been very use...","['two day bangkok useful productive', 'first b...",two day bangkok useful productive first bimste...


In [8]:
df_MS.shape

(1401, 6)

In [9]:
df_NM.shape

(1182, 6)

In [10]:
meta_df = pd.concat([df_NM, df_MS])

In [11]:
meta_df.shape


(2583, 6)

In [12]:
meta_df['Cleaned_Speech_string'] = meta_df['Cleaned_Speech_string'].astype(str)

In [13]:
corpus_tokenized = [nltk.word_tokenize(sentence) for sentence in meta_df["Cleaned_Speech_string"]]

In [14]:
len(corpus_tokenized)

2583

In [15]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
tagged_docs = [TaggedDocument(words=doc, tags=[i]) for i, doc in enumerate(corpus_tokenized)]

In [16]:
len(tagged_docs)

2583

In [17]:
#TRAINING THE MODEL OVER OUR DATA
model = Doc2Vec(tagged_docs, vector_size=300, window=5, min_count=5, workers=4, epochs=20)

In [18]:
import pandas as pd

# Create an empty DataFrame with the desired column names
df_data = pd.DataFrame(columns=['index', 'vector'])

# Iterate over the corpus_tokenized list and generate a vector for each element
for i, speech in enumerate(corpus_tokenized):
    vector = model.infer_vector(speech)
    # Store the index number and vector in a dictionary
    data = {'index': i, 'vector': vector}
    # Add the dictionary as a new row in the DataFrame
    df_data.loc[len(df_data)] = data


In [19]:
for i in range(300):
    col_name = f'dim_{i+1}'
    df_data[col_name] = 0.0

# Iterate over each row in the DataFrame and assign the ith element of the vector to the corresponding column
for i, row in df_data.iterrows():
    for j, element in enumerate(row['vector']):
        col_name = f'dim_{j+1}'
        df_data.at[i, col_name] = element

C:\Users\Yatika Arora\AppData\Local\Temp\ipykernel_11436\829503481.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_data[col_name] = 0.0
C:\Users\Yatika Arora\AppData\Local\Temp\ipykernel_11436\829503481.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_data[col_name] = 0.0
C:\Users\Yatika Arora\AppData\Local\Temp\ipykernel_11436\829503481.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns

In [20]:
df_data = df_data.drop('index', axis=1)
df_data = df_data.drop('vector', axis=1)

In [22]:
df_data

,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_291,dim_292,dim_293,dim_294,dim_295,dim_296,dim_297,dim_298,dim_299,dim_300
0,-0.747492,-1.966573,0.009028,2.589960,1.751558,-0.087287,-0.384523,2.469708,0.765102,0.519457,...,0.183592,1.966122,0.744150,-0.243863,1.056715,2.260541,-0.261603,-0.133897,0.767028,1.452642
1,0.226026,-1.588540,1.993117,0.921904,1.256267,1.110481,-1.433172,0.039003,-1.349682,-2.506248,...,0.248839,0.048878,-1.195855,-2.668763,-0.301690,1.381714,0.198986,-2.189520,2.304302,1.954711
2,-1.599256,-3.157645,2.029897,-2.522245,0.699439,-1.810817,1.027597,-0.445623,-0.594759,0.828694,...,0.454653,0.646074,2.713696,-1.855047,0.253971,2.459474,-0.200763,-1.160954,1.590378,0.179543
3,-1.920073,-0.773325,-0.566751,-1.025480,0.500693,-0.760738,-0.101830,1.389027,-0.547602,0.994040,...,0.515777,-1.198608,0.881802,-0.555725,2.169612,1.460371,0.066834,1.163674,0.842049,0.347057
4,-0.968319,0.085563,2.146590,-0.003943,0.837278,-1.947698,2.130507,-0.080145,-0.835451,1.171854,...,-0.529961,0.131872,-1.274559,-1.672770,-1.383726,2.181927,1.014986,-0.549342,-1.210311,0.110370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2578,-0.360140,0.332048,-0.188309,-0.223458,0.089483,1.071215,0.153387,0.083013,0.043474,1.388782,...,-0.131700,-0.263584,-0.415667,0.070747,0.205825,0.683875,-0.324322,-0.080512,0.861277,0.534516
2579,0.887477,0.866085,1.550984,0.380191,0.281372,-0.479572,-0.197695,0.474337,-0.408057,0.081027,...,-0.761216,-0.635963,0.658220,-1.451545,0.980950,0.767212,-0.192354,-0.625514,1.033354,-0.771505
2580,0.158844,0.115370,0.945081,-0.646863,0.499315,0.859286,0.309408,0.886351,-0.589272,0.014200,...,0.804110,-0.615642,0.501816,-0.344570,0.695208,2.090110,-0.734541,-0.635918,0.550356,0.371139
2581,0.188669,-0.235360,2.220449,-0.524353,-0.407597,0.652143,0.662465,-0.514405,0.623942,-0.779387,...,0.152170,-0.198071,1.273489,-0.719707,1.438383,0.511749,-1.916128,-1.208402,-0.722597,0.444926


In [25]:
meta_df

,index,Date,Speech,Title,Cleaned_Speech,Cleaned_Speech_string
0,NMS1,2017-01-03,"Governor of Andhra Pradesh, Shri E. S. L. Nara...",PM's Address at the Inauguration of the 104th ...,['governor andhra pradesh shri e l narasimhan ...,governor andhra pradesh shri e l narasimhan ch...
1,NMS2,2017-01-09,Chief Minister of Gujarat Shri Vijay Rupani ji...,Text of PM's address at the Inauguration of No...,['chief minister gujarat shri vijay rupani ji ...,chief minister gujarat shri vijay rupani ji co...
2,NMS3,2017-01-09,I am delighted to be here at Gift City to ina...,Text of PM’s address on the occasion of Inaugu...,['delighted gift city inaugurate india first i...,delighted gift city inaugurate india first int...
3,NMS4,2017-02-01,I congratulate the Finance Minister Arun Jaitl...,English rendering of the Prime Minister’s Stat...,['congratulate finance minister arun jaitley j...,congratulate finance minister arun jaitley jee...
4,NMS5,2017-02-05,Sri Pejavar Math’s most respected Sri Vishwesh...,English rendering of the text of PM’s address ...,['sri pejavar math respected sri vishwesh tirt...,sri pejavar math respected sri vishwesh tirth ...
...,...,...,...,...,...,...
1396,MSS1397,2014-02-21,"""Madam Speaker,\n\nAs we come to the end of th...",PM’s farewell speech to the 15th Lok Sabha,['madam speaker come end journey fifteenth lok...,madam speaker come end journey fifteenth lok s...
1397,MSS1398,2014-02-26,\n\t“Today marks a very important step forward...,PM's speech at the foundation laying ceremony ...,['today mark important step forward towards es...,today mark important step forward towards esta...
1398,MSS1399,2014-03-01,\n\t“I am very happy to participate in the Nat...,PM’s address at the National Awards Ceremony f...,['happy participate national award function mi...,happy participate national award function mini...
1399,MSS1400,2014-03-04,\n\t“I am delighted to return to this beautifu...,PM’s statement at 3rd BIMSTEC Summit,['delighted return beautiful city nay pyi taw ...,delighted return beautiful city nay pyi taw th...


In [27]:
df_data.insert(0, "index1", meta_df["index"])
df_data.insert(1, "Date1", meta_df["Date"])

ValueError: cannot reindex on an axis with duplicate labels

In [26]:
df_data

,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_291,dim_292,dim_293,dim_294,dim_295,dim_296,dim_297,dim_298,dim_299,dim_300
0,-0.747492,-1.966573,0.009028,2.589960,1.751558,-0.087287,-0.384523,2.469708,0.765102,0.519457,...,0.183592,1.966122,0.744150,-0.243863,1.056715,2.260541,-0.261603,-0.133897,0.767028,1.452642
1,0.226026,-1.588540,1.993117,0.921904,1.256267,1.110481,-1.433172,0.039003,-1.349682,-2.506248,...,0.248839,0.048878,-1.195855,-2.668763,-0.301690,1.381714,0.198986,-2.189520,2.304302,1.954711
2,-1.599256,-3.157645,2.029897,-2.522245,0.699439,-1.810817,1.027597,-0.445623,-0.594759,0.828694,...,0.454653,0.646074,2.713696,-1.855047,0.253971,2.459474,-0.200763,-1.160954,1.590378,0.179543
3,-1.920073,-0.773325,-0.566751,-1.025480,0.500693,-0.760738,-0.101830,1.389027,-0.547602,0.994040,...,0.515777,-1.198608,0.881802,-0.555725,2.169612,1.460371,0.066834,1.163674,0.842049,0.347057
4,-0.968319,0.085563,2.146590,-0.003943,0.837278,-1.947698,2.130507,-0.080145,-0.835451,1.171854,...,-0.529961,0.131872,-1.274559,-1.672770,-1.383726,2.181927,1.014986,-0.549342,-1.210311,0.110370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2578,-0.360140,0.332048,-0.188309,-0.223458,0.089483,1.071215,0.153387,0.083013,0.043474,1.388782,...,-0.131700,-0.263584,-0.415667,0.070747,0.205825,0.683875,-0.324322,-0.080512,0.861277,0.534516
2579,0.887477,0.866085,1.550984,0.380191,0.281372,-0.479572,-0.197695,0.474337,-0.408057,0.081027,...,-0.761216,-0.635963,0.658220,-1.451545,0.980950,0.767212,-0.192354,-0.625514,1.033354,-0.771505
2580,0.158844,0.115370,0.945081,-0.646863,0.499315,0.859286,0.309408,0.886351,-0.589272,0.014200,...,0.804110,-0.615642,0.501816,-0.344570,0.695208,2.090110,-0.734541,-0.635918,0.550356,0.371139
2581,0.188669,-0.235360,2.220449,-0.524353,-0.407597,0.652143,0.662465,-0.514405,0.623942,-0.779387,...,0.152170,-0.198071,1.273489,-0.719707,1.438383,0.511749,-1.916128,-1.208402,-0.722597,0.444926


In [28]:
meta_docvectors = open("meta_docvectors.pickle", "wb")
pickle.dump(df_data, meta_docvectors)
meta_docvectors.close()